# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# Описание

Необходимо разработать модель наиболее качественного и быстрого расчета стоимости автомобилей на основе исторических данных о технических характеристиках, комплектации и ценах автомобилей.

## Подготовка данных

1.1. Загрузим библиотеки и файлы с данными, которые нужны нам в работе.

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, KFold
from sklearn.metrics import mean_squared_error as mse
from math import sqrt
from catboost import CatBoostRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
import time
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from datetime import datetime 

In [2]:
data = pd.read_csv('/datasets/autos.csv')
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


1.2. Предобработка данных

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
# Удалим колонки не влияющие на целевой признак, соответственно ненужные для обучения моделей: DateCrawled, DateCreated, LastSeen (даты скачивания и создания объявления, дата последней активности пользователя),
# NumberOfPictures (количество фотографий автомобиля),PostalCode (почтовый индекс владельца анкеты (пользователя)

data.drop(columns = ['DateCrawled', 'DateCreated', 'LastSeen', 'NumberOfPictures', 'PostalCode'], inplace=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              354369 non-null  int64 
 1   VehicleType        316879 non-null  object
 2   RegistrationYear   354369 non-null  int64 
 3   Gearbox            334536 non-null  object
 4   Power              354369 non-null  int64 
 5   Model              334664 non-null  object
 6   Kilometer          354369 non-null  int64 
 7   RegistrationMonth  354369 non-null  int64 
 8   FuelType           321474 non-null  object
 9   Brand              354369 non-null  object
 10  Repaired           283215 non-null  object
dtypes: int64(5), object(6)
memory usage: 29.7+ MB


In [6]:
data.isna().sum() # посмотрим пропуски

Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
dtype: int64

In [7]:
# Заполним пропуски в категориальных данных текстом 'no information available'(строк с пропусками больше 10% от общего объема данных, поэтому удалить мы их не можем)
# восстановить данные по тем данным, что есть, мы не можем, так как пропуски в категориальных признаках
data = data.fillna('no information available')
data.isna().sum()

Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Kilometer            0
RegistrationMonth    0
FuelType             0
Brand                0
Repaired             0
dtype: int64

In [8]:
data.duplicated().sum() #проверим дубликаты


27543

In [9]:
data = data.drop_duplicates()  #дубликатов менее 10% от общего объема данных, соответственно можем их удалить без потерь для информативности выборки
data.duplicated().sum()

0

1.3.Посмотрим выбросы

In [10]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth
count,326826.000000,326826.000000,326826.000000,326826.000000,326826.000000
mean,4403.747330,2004.230985,110.244705,128144.073605,5.696239
std,4518.209808,91.120018,195.886373,37947.663920,3.724122
min,0.000000,1000.000000,0.000000,5000.000000,0.000000
25%,1000.000000,1999.000000,69.000000,125000.000000,3.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000
75%,6350.000000,2008.000000,141.000000,150000.000000,9.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000


In [11]:
data['Price'].value_counts()

0       9387
500     5305
1500    5013
1000    4299
1200    4243
        ... 
1910       1
1654       1
5748       1
7285       1
8188       1
Name: Price, Length: 3731, dtype: int64

In [12]:
data = data[data.Price > 100 ] # удалим строки с ценой, меньше "100", так как это нереалистичная цена, создает выбросы, а количество таких строк менее 10 %
data['Price'].value_counts()

500      5305
1500     5013
1000     4299
1200     4243
2500     4036
         ... 
6039        1
3988        1
14227       1
11920       1
8188        1
Name: Price, Length: 3673, dtype: int64

In [13]:
# в столбце RegistrationYear есть нереалистичные данные, исправим это:(машиностроение, как отрасль производства ведет историю с 1880-1890-х годов)
def Balance_RegistrationYear(value):
    if value > 2016:
        return 2016
    elif value < 1890:
        return 1890
    else:
        return value
data["RegistrationYear"] = data["RegistrationYear"].apply(Balance_RegistrationYear)

# в столбце RegistrationMonth есть нереалистичные данные, исправим это:(месяц не может быть нулевым)
def Balance_RegistrationMonth(value):
    if value < 1:
        return 1
    else:
        return value
data["RegistrationMonth"] = data["RegistrationMonth"].apply(Balance_RegistrationMonth)

# в столбце Power есть нереалистичные данные, исправим это:(лошадиные силы не могут быть меньше 9(для очень ретро-автомобилей, заменим на 9)
data.loc[data['Power'] < 9, 'Power'] = 9
# также не могут быть машины более мощные, чем самый мощный автомобиль (4515 л.с.), исправим это:
data.loc[data['Power'] > 4500, 'Power'] = 4500

In [14]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth
count,314059.000000,314059.000000,314059.000000,314059.000000,314059.000000
mean,4582.233287,2003.125677,110.967672,128399.838884,5.872438
std,4519.793083,7.401979,101.520120,37325.469309,3.550276
min,101.000000,1890.000000,9.000000,5000.000000,1.000000
25%,1200.000000,1999.000000,70.000000,125000.000000,3.000000
50%,2900.000000,2003.000000,105.000000,150000.000000,6.000000
75%,6500.000000,2008.000000,143.000000,150000.000000,9.000000
max,20000.000000,2016.000000,4500.000000,150000.000000,12.000000


# Вывод:

1. Мы удалили неинформативные и соответственно ненужные для обучения модели столбцы, чтобы сократить используюемую память, что в дальнейшем сократит скорость обучения);
2. удалили дубликаты;
3. выявили и заполнили пропуски текстом 'no information available', так как пропусков слишком много, чтобы удалить строки с пропусками, а заполнение пропусков на основании имеющихся данных весьма недостоверно, так как это категориальные признаки;
4. заменили нереалистичные данные в столбцах на более реальные;
5. в столбце price удалили данные с ценами меньше 100, как нереалистичные.

## Обучение моделей

2.1. Разделим выборки и подготовим кросс-валидатор для последующего обучения моделей

In [15]:
X = data.drop(columns=['Price'])
y = data.Price

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345)

In [16]:
# создадим кросс-валидатор для последующего обучения моделей
cv = KFold(n_splits=3, shuffle=True, random_state=12345)

2.2. Обучение моделей

In [37]:
# Выделим колонки, содержащие категориальные переменные
cat_columns = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']


# напишем функцию, принимающую модель, данные и возвращающую RMSE
# Добавил возврат значения RMSE на кросс-валидации
def model_elector(model, features_train, target_train, best_score):
    model.fit(features_train, target_train)
    predicted = model.predict(features_train)
    rmse = mse(target_train, predicted)**0.5
    cv_rmse = np.sqrt(-1 * best_score)
    return rmse, cv_rmse

А. Ridge

In [18]:
# Подготовимм пайплайн one-hot encoder -> scaler -> Ridge
pipe = Pipeline([
    ('ohe', OneHotEncoder(handle_unknown='ignore')),
    ('scaler', StandardScaler(with_mean=False)),
    ('model', Ridge(random_state=12345))
])

params = [
    {
        'model__alpha': np.logspace(-2, 3, 20)
    } 
]

In [19]:
grid_ridge = GridSearchCV(pipe,
                    param_grid=params,
                    cv=cv,
                    scoring='neg_mean_squared_error',
                    n_jobs=-1,
                    verbose=False)

In [20]:
%%time
grid_ridge.fit(X_train, y_train);

CPU times: user 36.7 s, sys: 3.43 s, total: 40.1 s
Wall time: 40.2 s


GridSearchCV(cv=KFold(n_splits=3, random_state=12345, shuffle=True),
             estimator=Pipeline(steps=[('ohe',
                                        OneHotEncoder(handle_unknown='ignore')),
                                       ('scaler',
                                        StandardScaler(with_mean=False)),
                                       ('model', Ridge(random_state=12345))]),
             n_jobs=-1,
             param_grid=[{'model__alpha': array([1.00000000e-02, 1.83298071e-02, 3.35981829e-02, 6.15848211e-02,
       1.12883789e-01, 2.06913808e-01, 3.79269019e-01, 6.95192796e-01,
       1.27427499e+00, 2.33572147e+00, 4.28133240e+00, 7.84759970e+00,
       1.43844989e+01, 2.63665090e+01, 4.83293024e+01, 8.85866790e+01,
       1.62377674e+02, 2.97635144e+02, 5.45559478e+02, 1.00000000e+03])}],
             scoring='neg_mean_squared_error', verbose=False)

In [21]:
grid_ridge.best_params_

{'model__alpha': 545.5594781168514}

In [33]:
%%time

target_predict = grid_ridge.best_estimator_.predict(X_train)

CPU times: user 402 ms, sys: 27.3 ms, total: 430 ms
Wall time: 436 ms


Б.LightGBM

In [23]:
# кодируем категориальные признаки через LabelEncoder и с помощью гридсёрча выберем наиболее подходящие параметры модели
encoder = LabelEncoder()
lgbm_train = X_train.copy()
lgbm_test = X_test.copy()

for col in cat_columns:
    lgbm_train[col] = encoder.fit_transform(lgbm_train[col])
    lgbm_test[col] = encoder.fit_transform(lgbm_test[col])
    
lgbm = LGBMRegressor(boosting_type='gbdt', random_state=12345)
params = {
    'learning_rate': np.logspace(-3, 0, 5),
    'n_estimators': [40, 60],
    'num_leaves': [31, 41, 51],
}

gs_gbm = GridSearchCV(lgbm,params, cv=cv, scoring='neg_mean_squared_error', verbose=True)

In [24]:
%%time
gs_gbm.fit(lgbm_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
CPU times: user 6h 41min 30s, sys: 7min 6s, total: 6h 48min 36s
Wall time: 6h 54min 24s


GridSearchCV(cv=KFold(n_splits=3, random_state=12345, shuffle=True),
             estimator=LGBMRegressor(random_state=12345),
             param_grid={'learning_rate': array([0.001     , 0.00562341, 0.03162278, 0.17782794, 1.        ]),
                         'n_estimators': [40, 60], 'num_leaves': [31, 41, 51]},
             scoring='neg_mean_squared_error', verbose=True)

In [25]:
gs_gbm.best_params_

{'learning_rate': 0.1778279410038923, 'n_estimators': 60, 'num_leaves': 51}

In [36]:
%%time

target_predict = gs_gbm.best_estimator_.predict(lgbm_train)

CPU times: user 1.34 s, sys: 6.98 ms, total: 1.35 s
Wall time: 1.35 s


В.catboost

In [27]:
cbr = CatBoostRegressor(random_seed=12345,
                        loss_function='RMSE',
                        silent=True,
                        cat_features=cat_columns)

params = {
    'learning_rate': np.logspace(-3, 0, 5),
    'iterations': [40, 60],
    'depth': [d for d in range(2, 11)],
}

grid_cbr = GridSearchCV(cbr,
                        params,
                        cv=cv,
                        scoring='neg_mean_squared_error',
                        verbose=False)

In [28]:
%%time
grid_cbr.fit(X_train, y_train);

CPU times: user 18min 3s, sys: 11.2 s, total: 18min 14s
Wall time: 21min 37s


GridSearchCV(cv=KFold(n_splits=3, random_state=12345, shuffle=True),
             estimator=<catboost.core.CatBoostRegressor object at 0x7f121ceb8ee0>,
             param_grid={'depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'iterations': [40, 60],
                         'learning_rate': array([0.001     , 0.00562341, 0.03162278, 0.17782794, 1.        ])},
             scoring='neg_mean_squared_error', verbose=False)

In [29]:
grid_cbr.best_params_

{'depth': 10, 'iterations': 60, 'learning_rate': 0.1778279410038923}

In [54]:
%%time

target_predict = grid_cbr.best_estimator_.predict(X_train)

CPU times: user 406 ms, sys: 4.13 ms, total: 410 ms
Wall time: 411 ms


# Вывод:

Мы обучили модели Ridge, CatBoost, LightGBM, посмотрели время обучения, а также время предсказаний на обучающей выборке.

## Анализ моделей

Используя нашу функцию и полученные данные сделаем табличку с показателями наших моделей

In [38]:
models = []
models.append(model_elector(grid_ridge.best_estimator_, X_train, y_train, grid_ridge.best_score_))
models.append(model_elector(grid_cbr.best_estimator_, X_train, y_train, grid_cbr.best_score_))
models.append(model_elector(gs_gbm.best_estimator_, lgbm_train, y_train, gs_gbm.best_score_))
our_models = pd.DataFrame(data=models, index=['Ridge', 'CatBoost', 'LightGBM'], columns=['RMSE', 'RMSE on CV'])

display(our_models)

,RMSE,RMSE on CV
Ridge,2016.875829,2043.491043
CatBoost,1661.540885,1716.818001
LightGBM,1663.178390,1712.799091


In [49]:
# добавим данные по скорости

our_models["trainig_time, сек"] = [45.1, 1297, 24864]
our_models["predict_time, сек"] = [0.436, 0.428, 1.35]

display(our_models)

,RMSE,RMSE on CV,"trainig_time, сек","predict_time, сек"
Ridge,2016.875829,2043.491043,45.1,0.436
CatBoost,1661.540885,1716.818001,1297.0,0.428
LightGBM,1663.178390,1712.799091,24864.0,1.350


# Вывод:

Лучший RMSE on CV у модели LightGBMб,почти такой же у CatBoost, лучшее время обучения - у модели Ridge. LightGBM  очень долго обучается. У модели CatBoost лучшее время предсказания.

## Тестирование лучшей модели

Проверим время предсказания и RMSE нашей лучшей модели на тестовой выборке

In [51]:
%%time

predictions_cbr_test = grid_cbr.best_estimator_.predict(X_test)

CPU times: user 101 ms, sys: 4.02 ms, total: 105 ms
Wall time: 159 ms


In [52]:
sqrt(mse(y_test, predictions_cbr_test)) 

1727.563443511245

# Вывод:
время предсказания небольшое, RMSE меньше 2500. Выбранная модель соответствует условиям нашей задачи.

# Общий вывод:

Мы разработали модель наиболее качественного и быстрого расчета стоимости автомобилей на основе исторических данных о технических характеристиках, комплектации и ценах автомобилей.

Для этого мы рассмотрели исторические данные, внесли в них необходимые для сокращения используемой памяти и корректности дальнейшего обучения моделей изменения.
Затем мы обучили и проанализировали качество и скорость обучения и предсказаний моделей Ridge, CatBoost, LightGBM.
По соотношению скорости и качества лидирует модель CatBoost. Проверили ее на тестовой выборке, что подтвердило наши выводы.

Основываясь на полученных результатах, предложим заказчику модель CatBoost, которая выдает наименьшую ошибку и приемлемое, хоть и не самое лучшее время обучения и предсказаний.